#### You can refer to the [commands](./commands.md) file for the list of commands utilized in this demo.

In [ ]:
# !$SPARK_HOME/sbin/start-master.sh
# !$SPARK_HOME/sbin/start-worker.sh <SPARK_URL>

In [1]:
import pandas as pd
import os
import json
from glob import glob
import pathlib
import uuid

### Check your data folder to ensure availability of files to process

In [2]:
os.listdir(f"/home/{os.environ.get('USER')}/setu-refactor/examples/sample_data/english")

['02448f577d0bcc8f438dd06fafada4d33711b361.json',
 '05854b4f9d0751b81403bf3436ed03559fa9c3ce.json',
 '07364b2b241fd42bee97cf7031f705467a29be7b.json',
 '072fe8ae637c266d6410b9d1d0657ecb3cc6c468.json',
 '066fa21773ec16f318d89478bbdc2608548f8d5f.json',
 '0606c1eb13b654fd61cb1e9be1c80a9fd4f01889.json',
 '0064da3e58e33153f4b415acbf1aa35c526c9657.json',
 '017964151c474e9f261922ba78f4fe8f29ecdb88.json',
 '0671ad9983db465c43c89344d419a81d9ca56ba4.json',
 '0318dc51a06f31e72074cf359b30f666ecbd5152.json',
 '053e749677b01684e2f20c31e3ba23426588519d.json',
 '037590d2a82f5efa532fc80519e9e022e901783e.json',
 '053e16c0b61aba25235662fc2717126b44a44702.json',
 '06951af315ca9b94e2a016bc4d6780c8a22d0e2a.json',
 '06800df8ebb9232796d5f0a9f7897bd3a4fee56a.json',
 '07305d21afe5335bf80f2a4ef5df474101e67865.json',
 '03070278f98cbb9bf9f94529f5c3d0d0666cd7a4.json',
 '0371f3ab284b050ce35fa156798e42408e35af24.json',
 '0200bd2bf9e071216c4e46802ef9bceef5ead1fd.json',
 '05230ba4e1461beb11c11771a45e9b7b508d9b28.json',


### Run the JSON2Parquet Stage to convert your data jsons into parquets

In [3]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    JSON2ParquetStage \
    --json_glob_path "/home/$USER/setu/examples/sample_data/english/*.json" \
    --language english \
    --j2p_samples_per_partition 1500 \
    --j2p_parquet_output_path /home/$USER/setu/examples/output/j2p_output

24/03/26 07:42:13 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/26 07:42:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/26 07:42:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Command used to run this script:  /home/shanks/setu/setu/run.py --config /home/shanks/setu/configs/crawls/spark_english_config.json --mode crawl --run_local True JSON2ParquetStage --json_glob_path /home/shanks/setu/examples/sample_data/english/*.json --language english --j2p_samples_per_partition 1500 --j2p_parquet_output_path /home/shanks/setu/examples/output/j2p_output
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
-------

### Check the JSON2Parquet Stage Output

In [ ]:
json_output_path = f"/home/{os.environ.get('USER')}/setu-refactor/examples/output/j2p_output"

In [ ]:
parquets = glob(f"{json_output_path}/*.parquet")

In [ ]:
df = pd.read_parquet(parquets[0])

In [ ]:
df.head()

### Run the Text Extraction State to extract text from the previous stage output

In [ ]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    ExtractTextStage \
    --te_parquets_path "/home/$USER/setu/examples/output/j2p_output/*.parquet" \
    --te_samples_per_partition 1500 \
    --te_run_mode data \
    --te_output_path "/home/$USER/setu/examples/output/te_output"


### Check the TextExtract Stage Output

In [ ]:
te_output_path = f"/home/{os.environ.get('USER')}/setu-refactor/examples/output/te_output"

In [ ]:
parquets = glob(f"{te_output_path}/*.parquet")

In [ ]:
parquets

In [ ]:
df = pd.read_parquet(parquets[0])

In [ ]:
df.head()

### Run DocCleanStage on the Text Extraction Stage output

In [ ]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    DocCleanStage \
    --doc_df_parquets_path "/home/$USER/setu/examples/output/te_output/*.parquet" \
    --is_doc_df_path_batched False \
    --doc_clean_additional_cols_to_use "url,source,language" \
    --use_symbol_filter True \
    --doc_clean_samples_per_partition 1500 \
    --doc_clean_verbose False \
    --doc_clean_run_mode data \
    --symbol_filter_output_path "/home/$USER/setu/examples/output/symbol_filter/" \
    --cleaned_doc_output_path "/home/$USER/setu/examples/output/cleaned_docs/" 

### Check DocClean Stage Output

In [ ]:
cd_output_path = f"/home/{os.environ.get('USER')}/setu-refactor/examples/output/cleaned_docs"

In [ ]:
parquets = glob(f"{cd_output_path}/*.parquet")

In [ ]:
df = pd.read_parquet(parquets[0])

In [ ]:
df.head()

### Run the LIDStage

In [ ]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 3 \
    --executor-cores 2 \
    --executor-memory 6G \
    --driver-memory 12G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    LIDStage \
    --lid_df_parquets_path "/home/$USER/setu/examples/output/cleaned_docs/*.parquet" \
    --is_lid_df_path_batched False \
    --lid_additional_cols "url,source,language" \
    --lid_samples_per_partition 1500 \
    --lid_verbose False \
    --lid_run_mode data \
    --doc_lid_output_path "/home/$USER/setu/examples/output/lid/"

### Check LIDStage Output 

In [ ]:
lid_output_path = f"/home/{os.environ.get('USER')}/setu-refactor/examples/output/lid/doc_lang_partition=english"

In [ ]:
parquets = glob(f"{lid_output_path}/*.parquet")

In [ ]:
df = pd.read_parquet(parquets[0])

In [ ]:
df.head()

### Run the Analysis Stages

In [ ]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    AnalysisStage \
    --analysis_df_parquets_path "/home/$USER/setu/examples/output/lid/*/*.parquet" \
    --is_analysis_df_path_batched False \
    --analysis_additional_cols_to_use "url,source,language,doc_lang,doc_lang_iso" \
    --analysis_samples_per_partition 1500 \
    --analysis_verbose False \
    --analysis_run_mode stage \
    --line_stats_output_path "/home/$USER/setu/examples/output/line_stats/" \
    --doc_stats_output_path "/home/$USER/setu/examples/output/doc_stats/" \
    --analysis_output_path "/home/$USER/setu/examples/output/analysis/"

### View Analysis Output

In [ ]:
al_output_path = f"/home/{os.environ.get('USER')}/setu-refactor/examples/output/doc_stats/doc_lang_partition=english"

In [ ]:
parquets = glob(f"{al_output_path}/*.parquet")

In [ ]:
parquets

In [ ]:
df = pd.read_parquet(parquets[0])

In [ ]:
df.head()

# Run the Flagging and Filtering Stage

In [ ]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    FlaggingAndFilteringStage \
    --doc_stats_parquets_path "/home/$USER/setu/examples/output/doc_stats/*/*.parquet" \
    --is_doc_stats_path_batched False \
    --fnf_samples_per_partition 1500 \
    --fnf_verbose False \
    --fnf_run_mode stage \
    --save_nsfw_data True \
    --nsfw_output_path "/home/$USER/setu/examples/output/nsfw/" \
    --filtered_doc_stats_output_path "/home/$USER/setu/examples/output/filtered_doc_stats/"

### View Filtered Document Statistics

In [ ]:
fd_output_path = f"/home/{os.environ.get('USER')}/setu-refactor/examples/output/filtered_doc_stats/"

In [ ]:
parquets = glob(f"{fd_output_path}/*.parquet")

In [ ]:
parquets

In [ ]:
df.head()

### Run Document Removal Stage

In [ ]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    DocumentRemovalStage \
    --analysis_out_path "/home/$USER/setu/examples/output/analysis/*/*.parquet" \
    --doc_stats_path "/home/$USER/setu/examples/output/doc_stats/*/*.parquet" \
    --doc_removal_join_col "doc_id" \
    --doc_removal_samples_per_partition 1500 \
    --doc_removal_verbose False \
    --doc_removal_run_mode stage \
    --filtered_docs_path "/home/$USER/setu/examples/output/filtered_doc_stats/"

## View Final Doc Stats output

In [ ]:
fd_output_path = f"/home/{os.environ.get('USER')}/setu-refactor/examples/output/filtered_doc_stats/"

In [ ]:
parquets = glob(f"{fd_output_path}/*.parquet")

In [ ]:
df = pd.read_parquet(parquets[0])

In [ ]:
df.head()

In [ ]:
df.info()

### Prepare for Deduplication

In [ ]:
columns = df.columns
columns

In [ ]:
new_columns = ['doc_id', 'url', 'source', 'language', 'doc_lang', 'doc_lang_iso',
       'text', 'bytes', 'words_count', 'char_count', 'lines_count',
       'mean_line_length', 'min_line_length', 'max_line_length',
       'nsfw_words_count', 'non_li_char_count']

In [ ]:
new_df = df[new_columns]

In [ ]:
new_df.to_parquet("/home/shanks/setu-refactor/examples/output/prededup/prededup.parquet")

#### Run Minhash Deduplication on the Filtered Docs output

In [4]:
!python ../text-dedup/text_dedup/minhash.py \
    --path "parquet" \
    --name "sangraha-english" \
    --split "train" \
    --data_files "/home/shanks/setu-refactor/examples/output/prededup/*.parquet" \
    --cache_dir "/home/shanks/cache" \
    --output "/home/$USER/setu/examples/output/minhash/" \
    --column "text" \
    --batch_size 10000

python: can't open file '/home/shanks/setu/examples/../text-dedup/text_dedup/minhash.py': [Errno 2] No such file or directory


### Load and View your final dataset

In [ ]:
from datasets import Dataset

ds = Dataset.from_file("/home/shanks/setu-refactor/examples/output/minhash/data-00000-of-00001.arrow")

In [ ]:
df = ds.to_pandas()

In [ ]:
df.head()